In [5]:
# poppler should be installed on your device to use pdf2image
from pdf2image import convert_from_path
import pytesseract

In [8]:
# CMakeのインストールが必要であり，筆者のMacではインストール不可であるため以下のコード実行を断念
# This code is generated with ChatGPT
# PDFを画像に変換
pages = convert_from_path('src/text_recognition/resource/蝸牛考_1823865_0001.pdf', dpi=300)

# 各ページをOCR処理
for i, page in enumerate(pages):
    # 画像を保存してOCR実行
    page.save(f'page_{i}.jpg', 'JPEG')
    text = pytesseract.image_to_string(page, lang='jpn', config='--psm 5')
    print(f"Page {i + 1}:\n{text}\n")

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [4]:
# This code is generated with ChatGPT
from PIL import Image
import pytesseract

# 画像ファイルのパス
image_path = 'resource/kagyuko.png'

# 画像を開く
image = Image.open(image_path)

# OCR処理（日本語+縦書きモード）
# `--psm 5` は縦書き向け設定
text = pytesseract.image_to_string(image, lang='jpn', config='--psm 5')

# 結果を出力
print("OCR結果:")
print(text)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

本テキストのOCRには，思いつく限り以下の2通りのアプローチがあると思う．
1. 文字の座標を認識してから，その文字一字一字について認識精度を上げる
2. 画像全体を機械学習にかけ，文章を抽出するように訓練する

とりうる方法としては1が学習精度を上げるのに貢献しそう．なぜならば，一字一字の認識精度を上げる方法では画像全体を学習させる時に比べて元データ（画像）と正解データ（文字情報）の対応がはっきりしているから．
まずは文字の位置を正確に認識させてから，それら一字一字について精度を上げることを考えた萌芽，少ない計算量でも精度を上げられそう．
→多分ChatGPTなどは2の方法を用いているのだろう．

In [8]:
import cv2
import pytesseract
from pytesseract import Output

# Ensure pytesseract points to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Adjust for your installation path

def recognize_text_and_draw_boxes(image_path, lang='jpn_vert'):
    # Read the image
    image = cv2.imread(image_path)

    # Convert to grayscale for better OCR accuracy
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform OCR with vertical Japanese language
    data = pytesseract.image_to_data(gray, lang=lang, output_type=Output.DICT)

    # Iterate over each detected text box
    for i in range(len(data['text'])):
        text = data['text'][i].strip()
        if text:  # Only draw boxes for detected text
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the image with bounding boxes
    cv2.imshow('Recognized Text with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Replace 'vertical_japanese_image.jpg' with your image path
image_path = 'resource/kagyuko.png'
recognize_text_and_draw_boxes(image_path)

ModuleNotFoundError: No module named 'cv2'